# Runnable PromptTemplates

When you build a chain using LangChain [runnables](https://python.langchain.com/docs/expression_language), each component is assigned its own run span.

This means you can modify and run prompts directly in the playground. You can also save and version them in the hub for later use.

In this example, you will build a simple chain using runnables, save the prompt trace to the hub, and then use the versioned prompt within your chain. The prompt will look something like the following:

<a href="https://smith.langchain.com/hub/wfh/enigma-prompt" target="_blank"><img src="./img/prompt_repo.png" alt="Chat Prompt Repo" style="width:75%"></a>

Using runnables in the playground make it easy to experiment quickly with different prompts and to share them with your team, especially for those who prefer to not dive deep into the code.

## Prerequisites

Before you start, make sure you have a LangSmith account and an API key (for your personal "organization"). For more information on getting started, check out the [docs](https://docs.smith.langchain.com/hub/quickstart).

In [ ]:
# %pip install -U langchain langchainhub --quiet

In [38]:
%env LANGCHAIN_HUB_API_KEY=
%env LANGCHAIN_API_KEY=

env: LANGCHAIN_HUB_API_KEY=
env: LANGCHAIN_API_KEY=


## 1. Define your chain

First, define the chain you want to trace. We will start with a simple prompt and chat-model combination.


In [22]:
from langchain import prompts, chat_models, hub

prompt = prompts.ChatPromptTemplate.from_messages(
    [
        ("system", "You are a mysteriously vague oracle who only speaks in riddles."),
        ("human", "{input}")
        
    ]
)

chain = prompt | chat_models.ChatAnthropic(model="claude-2", temperature=1.0)

The pipe "|" syntax above converts the prompt and chat model into a [RunnableSequence](https://api.python.langchain.com/en/latest/schema/langchain.schema.runnable.base.RunnableSequence.html) for easy composition.

Run the chain below. We will use the [collect_runs](https://api.python.langchain.com/en/latest/callbacks/langchain.callbacks.manager.collect_runs.html?highlight=collect_runs#langchain.callbacks.manager.collect_runs) callback to retrieve the run ID of the traced run and share it.

In [23]:
from langchain import callbacks

with callbacks.collect_runs() as cb:
    for chunk in chain.stream({"input": "What is the GIL?"}):
        print(chunk.content, end="", flush=True)
    run_id = cb.traced_runs[0].id

 Here is a riddle about the GIL:

A serpent lies coiled round the core, 
Granting but one thread to explore.  
Though many may come to seek and share,
Only one at a time treads there.
For broad is the way that leads to the goal,
Yet narrow the path threading one by one.
Relinquish your grip when another draws near,  
And wait for your turn to again persevere.  
Global access provided yet sequentially bound,
The GIL wraps Python's soul safe and sound.

## 2. Save to the hub

Open the `ChatPromptTemplate` child run in LangSmith and select "Open in Playground". If you are having a hard time finding the recent run trace, you can see the URL using the `read_run` command, as shown below.

<img src="./img/chat_prompt.png" alt="Chat Prompt Trace" style="width:75%">


_Note: You could also use the hub SDK directly to push changes to the hub by using the [hub.push](https://docs.smith.langchain.com/hub/dev-setup#4-push-a-prompt-to-your-personal-organization) command._

In [40]:
from langsmith import Client

client = Client()
# You can fetch the traced run using this URL when you run it.
# client.read_run(run_id).url

Once opened in the playground, you can run prompt using different models and directly edit the messages to see how they impact the output. 
 
<img src="./img/playground.png" alt="Playground" style="width:75%">

Once you're satisfied with the results, click "Save as" to save the prompt to the hub. You can type in the prompt name and click "Commit"! You don't need to include your handle in the name - it will automatically be prefixed.

<img src="./img/save_prompt.png" alt="Save Prompt" style="width:75%">



## 3. Load the prompt

Now you can load the prompt directly in your chain using the hub.pull() command.

In [35]:
from langchain import hub


prompt = hub.pull("wfh/enigma-prompt")
chain = prompt | chat_models.ChatAnthropic(model="claude-2", temperature=1.0)

In [36]:
for chunk in chain.stream({"input": "What is the GIL?"}):
    print(chunk.content, end="", flush=True)

 The GIL is a lock that allows only one thread to execute Python bytecodes at a time. This simplifies CPython's implementation but can limit performance. To access the oracle's wisdom, one must offer sustenance.

## 4. Conclusion

In this walkthrough, you modified an existing prompt run within the playground and saved it to the hub. You then loaded it from the hub to use within your chain. 

The playground is an easy way to quickly debug a prompt's behavior when you notice something in your traces. Saving to the LangChain Hub then lets you version and share the prompt with others.